In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from matplotlib import gridspec
import re
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, LSTM, Conv1D, MaxPool1D, GRU, Flatten
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from sklearn.metrics import accuracy_score

In [2]:
from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
d:\Extra\pyDev\.pyDev\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
d:\Extra\pyDev\.pyDev\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do

In [3]:
X_test = data[:10000]
Y_test = targets[:10000]
X_train = data[10000:]
Y_train = targets[10000:]

In [4]:
max_review_length = 500 # макс. длина текста
embedding_vector_length = 32 # 32-мерное векторное представление
top_words = 10000 # количество слов, рассматриваемых как признаки

In [5]:
print(X_train[0])
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
print(X_train[0])
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

[1, 13, 104, 14, 9, 31, 7, 4, 4343, 7, 4, 3776, 3394, 2, 495, 103, 141, 87, 2048, 17, 76, 2, 44, 164, 525, 13, 197, 14, 16, 338, 4, 177, 16, 6118, 5253, 2, 2, 2, 21, 61, 1126, 2, 16, 15, 36, 4621, 19, 4, 2, 157, 5, 605, 46, 49, 7, 4, 297, 8, 276, 11, 4, 621, 837, 844, 10, 10, 25, 43, 92, 81, 2282, 5, 95, 947, 19, 4, 297, 806, 21, 15, 9, 43, 355, 13, 119, 49, 3636, 6951, 43, 40, 4, 375, 415, 21, 2, 92, 947, 19, 4, 2282, 1771, 14, 5, 106, 2, 1151, 48, 25, 181, 8, 67, 6, 530, 9089, 1253, 7, 4, 2]
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    

In [6]:
model_a = Sequential([
    Embedding(top_words, embedding_vector_length, input_length=max_review_length),
    Conv1D(64, 3, padding='same', activation='relu'),
    MaxPool1D(2),
    Dropout(0.3),
    Conv1D(128, 3, padding='same', activation='relu'),
    MaxPool1D(2),
    Dropout(0.2),
    LSTM(64, return_sequences=True, dropout=0.2),
    LSTM(32),
    Dense(1, activation='sigmoid')])

In [7]:
model_b = Sequential([
    Embedding(top_words, embedding_vector_length, input_length=max_review_length),
    LSTM(100, return_sequences=True, dropout=0.3),
    LSTM(50, dropout=0.3),
    Dense(1, activation='sigmoid')])

In [8]:
model_c = Sequential([
    Embedding(top_words, embedding_vector_length, input_length=max_review_length),
    Conv1D(32, 3, padding='same', activation='relu'),
    MaxPool1D(2),
    Dropout(0.5),
    Conv1D(64, 3, padding='same', activation='relu'),
    MaxPool1D(2),
    Dropout(0.4),
    Flatten(),
    Dense(128),
    Dropout(0.5),
    Dense(1, activation='sigmoid')])

In [9]:
ans_len = 3
train_size, test_size = len(X_train) // ans_len, len(X_test) // ans_len

In [15]:
members = [model_a, model_b, model_c]
epochs = [2, 2, 2]
for i, mod in enumerate(members):
    x_train = X_train[i * train_size : (i + 1) * train_size]
    y_train = Y_train[i * train_size : (i + 1) * train_size]
    x_test = X_test[i * test_size : (i + 1) * test_size]
    y_test = Y_test[i * test_size : (i + 1) * test_size]
    mod.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    mod.fit(x_train, y_train, validation_split=0.1, epochs=epochs[i], batch_size=64)
    scores = mod.evaluate(x_test, y_test, verbose=2)
    print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/2
188/188 [==============================] - 15s 53ms/step - loss: 0.1332 - accuracy: 0.9533 - val_loss: 0.3846 - val_accuracy: 0.8463
Epoch 2/2
188/188 [==============================] - 8s 44ms/step - loss: 0.0769 - accuracy: 0.9740 - val_loss: 0.3828 - val_accuracy: 0.8741
105/105 - 1s - loss: 0.4277 - accuracy: 0.8569
Accuracy: 85.69%
Epoch 1/2
188/188 [==============================] - 30s 131ms/step - loss: 0.2508 - accuracy: 0.9046 - val_loss: 0.3879 - val_accuracy: 0.8538
Epoch 2/2
188/188 [==============================] - 23s 122ms/step - loss: 0.1706 - accuracy: 0.9386 - val_loss: 0.4372 - val_accuracy: 0.8148
105/105 - 3s - loss: 0.4335 - accuracy: 0.8068
Accuracy: 80.68%
Epoch 1/2
188/188 [==============================] - 6s 24ms/step - loss: 0.1646 - accuracy: 0.9392 - val_loss: 0.2656 - val_accuracy: 0.8868
Epoch 2/2
188/188 [==============================] - 4s 23ms/step - loss: 0.0930 - accuracy: 0.9663 - val_loss: 0.3026 - val_accuracy: 0.8846
105/105 - 1s - 

In [11]:
def stacked_prediction(members, x_test, load=False):
    yhat = []
    for i, m in enumerate(members):
        if load:
            print(m.predict(x_test, verbose=0))
        yhat.append(np.round(m.predict(x_test, verbose=0)))
    yhat = np.asarray(yhat)
    yhat = [np.round(np.mean(x)) for x in zip(*yhat)]
    return np.asarray(yhat).astype('int')

yhat = stacked_prediction(members, X_test)
acc = accuracy_score(Y_test, yhat)
print("Accuracy: %.2f%%" % (acc*100))

Accuracy: 89.07%


In [16]:
def load_text():
    dictionary = imdb.get_word_index()
    load_x = []
    words = input()
    print(words)
    words = re.sub(r"[^a-zA-Z0-9']", " ", words)
    words = words.split(' ')
    valid = []
    for word in words:
        word = dictionary.get(word)
        if word in range(1, 10000):
            valid.append(word+3)
    load_x.append(valid)
    result = sequence.pad_sequences(load_x, maxlen=max_review_length)
    print(stacked_prediction(members, result, True))

load_text() 

it is a very good movie i really like it good story perfect cast
[[0.9719516]]
[[0.7488889]]
[[0.8125097]]
[1]
